# Eurybia - Overview
This tutorial will help you understand how Eurybia works with a simple use case

Contents:
- Compile Eurybia 
- Generate report

For a more detailed tutorial on :
- Data validation : (https://github.com/MAIF/eurybia/tree/master/tutorial/data_validation)
- Data drift : (https://github.com/MAIF/eurybia/tree/master/tutorial/data_drift)
- Model drift : (https://github.com/MAIF/eurybia/tree/master/tutorial/model_drift)

**Requirements notice** : the following tutorial may use third party modules not included in Eurybia.  
You can find them all in one file [on our Github repository](https://github.com/MAIF/eurybia/blob/master/requirements.dev.txt) or you can manually install those you are missing, if any.

In [1]:
from category_encoders import OrdinalEncoder
from lightgbm import LGBMRegressor
from eurybia import SmartDrift
from sklearn.model_selection import train_test_split

/Users/78176D/workspace/eurybia/eurybia/report/generation.py:18: UserWarning:

Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.



## Import Dataset and split in training and production dataset

In [2]:
from eurybia.data.data_loader import data_loading
house_df, house_dict = data_loading('house_prices')

In [3]:
# Let us consider that the column "YrSold" corresponds to the reference date. 
#In 2006, a model was trained using data. And in 2007, we want to detect data drift on new data in production to predict
#house price
house_df_learning = house_df.loc[house_df['YrSold'] == 2006]
house_df_2007 = house_df.loc[house_df['YrSold'] == 2007]

In [4]:
y_df_learning=house_df_learning['SalePrice'].to_frame()
X_df_learning=house_df_learning[house_df_learning.columns.difference(['SalePrice','YrSold'])]

y_df_2007=house_df_2007['SalePrice'].to_frame()
X_df_2007=house_df_2007[house_df_2007.columns.difference(['SalePrice','YrSold'])]

## Building Supervized Model

In [5]:
from category_encoders import OrdinalEncoder

categorical_features = [col for col in X_df_learning.columns if X_df_learning[col].dtype == 'object']

encoder = OrdinalEncoder(
    cols=categorical_features,
    handle_unknown='ignore',
    return_df=True).fit(X_df_learning)

X_df_learning_encoded=encoder.transform(X_df_learning)

In [6]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X_df_learning_encoded, y_df_learning, train_size=0.75, random_state=1)

In [7]:
regressor = LGBMRegressor(n_estimators=200).fit(Xtrain,ytrain)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001133 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 235, number of used features: 53
[LightGBM] [Info] Start training from score 181743.472340
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

## Use Eurybia for data drift

In [8]:
from eurybia import SmartDrift

In [13]:
X_df_2007["MasVnrType"] = X_df_2007["MasVnrType"].fillna("NR")
X_df_learning["MasVnrType"] = X_df_learning["MasVnrType"].fillna("NR")

SD = SmartDrift(df_current=X_df_2007,
                df_baseline=X_df_learning,
                deployed_model=regressor, # Optional: put in perspective result with importance on deployed model
                encoding=encoder, # Optional: if deployed_model and encoder to use this model
                dataset_names=("2007 dataset","Learning dataset") # Optional: Names for outputs
               )

In [14]:
%time SD.compile()

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x11fa079d0>
CPU times: user 13.6 s, sys: 3.54 s, total: 17.2 s
Wall time: 3.68 s


In [15]:
SD.generate_report(    
    output_file='report_house_price_datadrift_2007.html',    
    title_story="Data drift",
    title_description="""House price Data drift 2007""", # Optional: add a subtitle to describe report
    project_info_file="../eurybia/data/project_info_house_price.yml" # Optional: add information on report
    )